# CLASIFICACIÓN DE TWETS

En este libro analizaremos tweets, los cuales se buscarán que sean sobre los debates de las elecciones colombianas, para ver que tipo de usuarios existen en twitter y se tratará de clasificar los comentarios.

Primero que todo obtuvimos unos cuantos twets por medio de la herramienta (twint)[https://github.com/haccer/twint], con la cual podemos obtener discusiones alrededor de hashtags.

nuestro archivo se llamara output.csv.

Cargaremos el archivo con pandas

In [1]:
import pandas as pd
import numpy as np
datos = pd.read_csv('output.csv', header = 0)

Ahora vamos a ver como se estructuran los datos

In [3]:
print(datos.ix[0:4])

                   id        date      time timezone             user_id  \
0  991850911835226112  2018-05-02  01:24:19      GMT          1713348180   
1  991768558282604545  2018-05-02  19:57:05      GMT  876316397332307968   
2  991755106751340547  2018-05-02  19:03:38      GMT          1713348180   
3  991527191681945600  2018-05-01  03:57:58      GMT           438683266   
4  991499060073914368  2018-05-01  02:06:11      GMT           138093954   

       username                                              tweet  replies  \
0    dinativa86  @MoralesViviane @IvanDuque @JulyTrsplcs @adana...        0   
1  JuanEGalvis1  A todos los colombianos y votantes, POR FAVOR ...        0   
2    dinativa86  @IvanDuque @Olgallego El mismo que en #ElPaisD...        0   
3   Lorenssssss  #YoVotoFajardo Por un país ni de izquierda ni ...        0   
4         mmpa9  @Colcheck tienen el chequeo de las respuestas ...        0   

   retweets  likes  location                                        

Aqui podemos ver todos los atributos, de los cuales solo nos importan los twets

In [5]:
tweets = datos['tweet']
print(tweets.ix[0:5])

#reviso el tipo de dato de un tweet
print(type(tweets[0]))

0    @MoralesViviane @IvanDuque @JulyTrsplcs @adana...
1    A todos los colombianos y votantes, POR FAVOR ...
2    @IvanDuque @Olgallego El mismo que en #ElPaisD...
3    #YoVotoFajardo Por un país ni de izquierda ni ...
4    @Colcheck tienen el chequeo de las respuestas ...
5    Por diferente que sea tu opinión, por pequeño ...
Name: tweet, dtype: object
<class 'str'>


In [6]:
import spacy
print(tweets[0])
print(tweets[1])
nlp = spacy.load('es')

doc1 = nlp(tweets[0])
doc2 = nlp(tweets[1])

@MoralesViviane @IvanDuque @JulyTrsplcs @adanagueo Pero si duque en #ElPaisDeLosJovenes  dijo que creía en la vida desde la concepción pero esta deacuerdo con el aborto en los tres casos que despenaliza la ley, osea que para el, esta bien matar. Yo creo que eso es peor que estar de acuerdo con el aborto creyendo que no hay vida.
A todos los colombianos y votantes, POR FAVOR no voten por duque por miedo a petro, o por petro por miedo a Duque.Vote a conciencia y por el que considere su mejor opción.#EleccionesColombia #ElPaisDeLosJovenes


ahora lo que vamos a hacer es verificar que tan sililares son el primer y segundo tweet de los tweets que tenemos hasta ahora.

In [7]:
print(doc1.similarity(doc2))
print(doc2.similarity(doc1))

0.6687402392079549
0.6687402392079549


Ahora se creará un arreglo que sera NxN siendo N el numero de tweets que tengamos y este arreglo tenga la comparacion de similaridad de todos contra todos.

In [8]:
NroTweets = len(tweets)
print(NroTweets)
prueba = 10
Msimilaridad = np.zeros((NroTweets, NroTweets))
twToken = [nlp(tw) for tw in tweets]

3660


In [9]:

for i in range(NroTweets):
    for j in range(NroTweets):
        Msimilaridad[i][j] = twToken[i].similarity(twToken[j])


In [10]:
print(Msimilaridad)

[[1.         0.66874024 0.8689584  ... 0.34080931 0.62231225 0.7175055 ]
 [0.66874024 1.         0.75646423 ... 0.54287942 0.48631865 0.69787063]
 [0.8689584  0.75646423 1.         ... 0.43549057 0.61666425 0.6998623 ]
 ...
 [0.34080931 0.54287942 0.43549057 ... 1.         0.49552691 0.59214324]
 [0.62231225 0.48631865 0.61666425 ... 0.49552691 1.         0.56815356]
 [0.7175055  0.69787063 0.6998623  ... 0.59214324 0.56815356 1.        ]]


Esto lo veremos como una matriz de distancias, ahora lo que deseamos es crear grupos dependiendo de esas distancias, grupos en los cuales los tweets tengan similaridades mayores del 95%, por empezar usaremos un diccionario de la siguiente manera y validaremos que los tweets sean un poco parecidos.

In [53]:
Grupos = {}
for tw in range(NroTweets):
    Grupos[tweets[tw]] = []
    for cmptw in range(NroTweets):
        if Msimilaridad[tw][cmptw] >= 0.90:
            if tw != cmptw:
                Grupos[tweets[tw]].append(tweets[cmptw])
            

Ahora compararemos par de grupos para ver que tanta conexion existe

In [54]:
cant = 0
for tw in range(NroTweets):
    if len(Grupos[tweets[tw]]) > 0:
        cant += 1

print(cant)



907


con esta forma de validar vemos que solo hay 236 tweets que tienen relación con otros, vamos a ver un par de tweets para ver un poco esto.

In [55]:
grupo = 0
for tw in range(NroTweets):
    if len(Grupos[tweets[tw]]) > 0:
        grupo += 1
        print('grupo: ', grupo)
        print(tweets[tw])
        print(Grupos[tweets[tw]][1])
        
    
    if grupo == 2:
        break
        
        

grupo:  1
#YoVotoFajardo Por un país ni de izquierda ni de derecha #ElPaisDeLosJovenes <Emoji: Nerd face><Emoji: Bar chart><Emoji: Books><Emoji: Pencil><Emoji: Memo><Emoji: Chart with upwards trend>
#ElPaísDeLosJóvenes Que Grande Eres @sergio_fajardo, un candidato completo e idóneo para sacar a <Emoji: Flag of Colombia> adelante sin corrupción y más educación <Emoji: Heavy check mark>
grupo:  2
#SiSeCumpliera seria una #Colombia con #LaFuerzaDeLaEsperanza que le da a un #Profesor para enseñar en #ElPaisDeLosJovenes que #SePuede https://twitter.com/fecode/status/991414473192861696 …
Despues que lo llamó #Bronx de #Colombia o que al #Bronx de #Bogotá lo llamó el #Catatumbo hace tres años? Ni #Coscorrones ni #titere menos un #populista #ExAlcalde #EleccionesColombia conteo regresivo para que se acabe esto #ColombiaDecide este es #ElPaisDeLosJovenes #SePuede https://twitter.com/JoseMAcevedo/status/988759754054012928 …


Ahora lo que haremos será usar un algoritmo de clasificación de la libreria sklearn , que se llama KMEANS, para crear centroides y que los tweets se ubiquen alrededor de estos.